In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoGRU
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.core import NeuralForecast
from neuralforecast.models import RNN

from datetime import datetime, timedelta
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from neuralforecast.losses.pytorch import MQLoss
from tqdm import tqdm

In [2]:
# Load and process the MEASLES_ARIZONA data
df = pd.read_csv("outbreaks_disease_location.csv")
value_columns = [str(i) for i in range(60)]
series_values = df[value_columns].fillna(0).astype(float)
start_dates = pd.to_datetime(df["start_date"])

# Shuffle and split
shuffled_indices = df.sample(frac=1, random_state=42).index
split_point = int(0.8 * len(df))
train_indices = shuffled_indices[:split_point]
test_indices = shuffled_indices[split_point:]

In [3]:
train_records = []
for i, row in series_values.iloc[train_indices].iterrows():
    dates = pd.date_range(start="2000-01-01", periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        train_records.append({"unique_id": f"Y{i+1}", "ds": dates[t], "y": value})
df_train = pd.DataFrame(train_records)

test_start_dates = start_dates.loc[test_indices] - pd.Timedelta(weeks=4)
df_test_all = []

for idx in test_indices:
    start_date = test_start_dates.loc[idx]
    row = series_values.loc[idx]
    dates = pd.date_range(start=start_date, periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        df_test_all.append({"unique_id": f"Y_{idx}", "ds": dates[t], "y": value})

df_test_all = pd.DataFrame(df_test_all)

In [4]:
df_train["date"] = pd.to_datetime(df_train["ds"])
df_train.set_index("date", inplace = True)

df_test_all["date"] = pd.to_datetime(df_test_all["ds"])
df_test_all.set_index("date", inplace = True)

In [5]:
df_test_all

,unique_id,ds,y
date,,,
2022-08-13,Y_5607,2022-08-13,20775.0
2022-08-20,Y_5607,2022-08-20,16570.0
2022-08-27,Y_5607,2022-08-27,13995.0
2022-09-03,Y_5607,2022-09-03,12145.0
2022-09-10,Y_5607,2022-09-10,10686.0
...,...,...,...
1939-05-20,Y_7270,1939-05-20,0.0
1939-05-27,Y_7270,1939-05-27,0.0
1939-06-03,Y_7270,1939-06-03,0.0


In [6]:
class FixedModelGRUProcessor:
    def __init__(self, dates=[]):
        self.dates  = dates
        self.forecasts, self.eval_pairs = [], []
        self.testset = None
        self.nf, self.model = None, None
        self.selected_input_size = None
        self.maes, self.mses, self.mapes, self.nmses = [], [], [], []
        self.metrics_df  = pd.DataFrame(columns=["Unique_id","Reference Date","MAE","MSE","MAPE","NMSE"])
        self.display_df  = pd.DataFrame(columns=["Unique_id","Reference Date","Target End Date","GT","Quantile","Prediction"])
        self.wis_df      = pd.DataFrame()

    # ──────────────────────────────────────────
    def create_fixed_model(self, df_train, df_test, h: int, freq: str, level=[]):
        min_len   = df_train.groupby("unique_id").size().min()
        max_inp   = max(8, min_len - h - 1)
        self.testset = df_test

        def cfg(t):
            return {
                "input_size"          : t.suggest_int("input_size", 8, max_inp),
                "encoder_hidden_size" : t.suggest_categorical("encoder_hidden_size",[32,64,128]),
                "encoder_n_layers"    : t.suggest_int("encoder_n_layers",1,3),
                "decoder_hidden_size" : t.suggest_categorical("decoder_hidden_size",[32,64,128]),
                "learning_rate"       : t.suggest_float("learning_rate",1e-4,1e-2,log=True),
                "batch_size"          : t.suggest_categorical("batch_size",[16,32,64]),
                "random_seed"         : t.suggest_int("random_seed",1,99999),
                "start_padding_enabled": True,
                "accelerator":"gpu","devices":2,"strategy":"auto"
            }

        model = AutoGRU(h=h, backend="optuna",
                        loss=MQLoss(level=level) if level else None,
                        config=cfg)

        self.nf = NeuralForecast(models=[model], freq=freq)
        self.nf.fit(df=df_train)
        self.model = self.nf.models[0].model
        self.selected_input_size = self.model.hparams["input_size"]

    # ──────────────────────────────────────────
    def sliding_batch_by_index(self, h: int = 4):
        uid_dates = {u: g["ds"].sort_values().to_numpy()
                     for u, g in self.testset.groupby("unique_id")}
        T, s = min(len(d) for d in uid_dates.values()), self.selected_input_size
        out = []

        for t in tqdm(range(s, T - h + 1), desc="index t"):
            batch, refs = [], []
            for uid, dates in uid_dates.items():
                win, ref = dates[t-s:t], dates[t-1]
                df_u = self.testset[self.testset["unique_id"] == uid]
                df_in = df_u[df_u["ds"].isin(win)].copy()
                if len(df_in) == s:
                    df_in["unique_id"] = uid
                    batch.append(df_in); refs.append((uid, ref))
            if not batch: continue

            fcst = self.nf.predict(df=pd.concat(batch)).set_index("ds")
            for uid, ref in refs:
                f  = fcst[fcst["unique_id"] == uid].reset_index()
                gt = (self.testset[self.testset["unique_id"] == uid]
                      .set_index("ds").loc[f["ds"]].reset_index())
                merged = pd.merge(f, gt, on=["unique_id","ds"], how="inner")
                merged["Reference Date"] = pd.to_datetime(ref).strftime("%Y-%m-%d")
                out.append(merged)

        return pd.concat(out, ignore_index=True)

    # ──────────────────────────────────────────
    def create_display_df(self, merged_df, model_name="AutoGRU"):
        recs = []
        for _, g in tqdm(merged_df.groupby("unique_id"), desc="display"):
            uid = g["unique_id"].iloc[0]; ref = g["Reference Date"].iloc[0]
            for col in g.columns:
                if not col.startswith(model_name): continue
                if col.endswith("median") or col == model_name:
                    q = 0.5
                elif "-lo-" in col:
                    lvl = int(col.split("-")[-1]); q = (1-lvl/100)/2
                elif "-hi-" in col:
                    lvl = int(col.split("-")[-1]); q = 1-(1-lvl/100)/2
                else:
                    continue
                tmp = g[["ds", col, "y"]].rename(columns={"ds":"Target End Date",
                                                           col:"Prediction",
                                                           "y":"GT"})
                tmp["Unique_id"] = uid
                tmp["Reference Date"] = ref
                tmp["Quantile"] = q
                recs.append(tmp)
        self.display_df = (pd.concat(recs)
                           .loc[:,["Unique_id","Reference Date","Target End Date",
                                   "GT","Quantile","Prediction"]]
                           .sort_values(["Unique_id","Reference Date",
                                         "Target End Date","Quantile"])
                           .reset_index(drop=True))

    # ──────────────────────────────────────────
    def efficient_compute_wis(self):
        if self.display_df.empty:
            raise ValueError("Run create_display_df first.")
        rows, df = [], self.display_df.sort_values(
            ["Unique_id","Reference Date","Target End Date","Quantile"])
        for (uid, ref, tgt), grp in tqdm(df.groupby(
                ["Unique_id","Reference Date","Target End Date"]), desc="WIS"):
            gt = grp["GT"].iloc[0]
            preds = grp.set_index("Quantile")["Prediction"]
            if 0.5 not in preds.index: continue
            rows.append({"Unique_id":uid,"Reference Date":ref,
                         "Target End Date":tgt,"GT":gt,
                         "WIS": abs(preds[0.5]-gt)})   # AE-only shortcut
        self.wis_df = pd.DataFrame(rows)

In [7]:
test_ids = [f"Y_{i}" for i in test_indices]

In [8]:
processor = FixedModelGRUProcessor()

In [9]:
processor.create_fixed_model(df_train, df_test_all, h=4, freq="W-SAT", level = [10,20,30,40,50,60,70,80, 85,90,95])

[I 2025-06-25 17:55:28,394] A new study created in memory with name: no-name-9281e1b1-6ae6-4a76-9649-3b7f80d473bc
[rank: 0] Seed set to 8702
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MQLoss        | 23     | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.


In [10]:
results = []

In [11]:
processor.selected_input_size

17

In [12]:
all_results = processor.sliding_batch_by_index(h=4)

index t:   0%|          | 0/40 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:   2%|▎         | 1/40 [00:34<22:34, 34.72s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:   5%|▌         | 2/40 [01:08<21:39, 34.19s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

index t:  95%|█████████▌| 38/40 [21:37<01:08, 34.03s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:  98%|█████████▊| 39/40 [22:11<00:33, 33.95s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t: 100%|██████████| 40/40 [22:45<00:00, 34.14s/it]


In [14]:
all_results.to_csv('GRU_concatenated.csv')

In [15]:
all_results

,ds,unique_id,AutoGRU-median,AutoGRU-lo-95,AutoGRU-lo-90,AutoGRU-lo-85,AutoGRU-lo-80,AutoGRU-lo-70,AutoGRU-lo-60,AutoGRU-lo-50,...,AutoGRU-hi-40,AutoGRU-hi-50,AutoGRU-hi-60,AutoGRU-hi-70,AutoGRU-hi-80,AutoGRU-hi-85,AutoGRU-hi-90,AutoGRU-hi-95,y,Reference Date
0,2011-01-15,Y_1,3.579459,0.705543,1.255663,1.579138,1.799896,2.086783,2.367707,2.583943,...,4.531922,4.899620,5.349906,5.970769,7.095424,8.038419,9.494263,11.914095,3.68098,2011-01-08
1,2011-01-22,Y_1,3.717173,1.037062,1.521978,1.816447,2.014087,2.274380,2.524569,2.744620,...,4.743606,5.127528,5.596482,6.269670,7.441012,8.439149,9.961667,12.390897,4.43182,2011-01-08
2,2011-01-29,Y_1,4.270710,1.193847,1.676227,1.979869,2.176026,2.473816,2.735508,3.022645,...,5.674232,6.161711,6.789306,7.693137,9.217614,10.498666,12.400012,14.991322,8.24176,2011-01-08
3,2011-02-05,Y_1,6.257584,1.139708,1.778094,2.183815,2.485344,2.968032,3.421597,3.892192,...,8.814830,9.762077,10.840039,12.468316,15.082901,17.007416,19.868940,22.908037,6.80219,2011-01-08
4,2021-09-11,Y_1000,15951.063477,-784.270264,452.750122,1435.529907,2176.018311,3653.205811,5290.423340,7022.626953,...,25699.378906,29416.160156,33243.476562,39051.746094,48261.363281,53912.160156,62899.367188,70539.234375,89095.00000,2021-09-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345595,1955-04-16,Y_9998,-0.013484,-1.683573,-1.271643,-0.994148,-0.791988,-0.537868,-0.383325,-0.259828,...,0.148143,0.177007,0.247073,0.381994,0.558603,0.677016,0.862966,1.303197,0.00000,1955-03-19
345596,1957-03-02,Y_9999,0.939883,0.675310,0.831585,0.875225,0.885632,0.890811,0.908788,0.928583,...,0.969259,0.980470,0.996907,1.038084,1.090945,1.111149,1.225382,1.483540,0.00000,1957-02-23
345597,1957-03-09,Y_9999,0.969965,0.760736,0.904867,0.934743,0.945290,0.942969,0.954806,0.965014,...,0.988773,0.995680,1.001867,1.031474,1.064275,1.073239,1.142447,1.345711,0.00000,1957-02-23
345598,1957-03-16,Y_9999,0.988516,0.812619,0.946812,0.966163,0.977719,0.973773,0.980508,0.985818,...,1.000300,1.003818,1.003386,1.022980,1.044869,1.047772,1.085189,1.252224,0.00000,1957-02-23


In [16]:
df = all_results.rename(columns={
    "ds"        : "Target End Date",
    "unique_id" : "Unique_id",
    "y"         : "GT"
})

# long‐format all quantile columns (everything that starts with AutoNBEATS-)
quantile_cols = [c for c in df.columns
                 if c.startswith("AutoGRU-")]

df_long = df.melt(
    id_vars = ["Unique_id", "Reference Date", "Target End Date", "GT"],
    value_vars = quantile_cols,
    var_name = "QuantileLabel",
    value_name = "Prediction"
)

def to_quantile(label: str) -> float:
    if "median" in label: return 0.5
    if "-lo-" in label:
        lvl = int(label.split("-")[-1]); return (1 - lvl/100) / 2
    if "-hi-" in label:
        lvl = int(label.split("-")[-1]); return 1 - (1 - lvl/100) / 2
    raise ValueError(label)

df_long["Quantile"] = df_long["QuantileLabel"].apply(to_quantile)

rows = []

group_iter = df_long.groupby(["Unique_id", "Reference Date", "Target End Date"])
for (uid, ref_date, tgt_date), g in tqdm(group_iter,
                                         total=len(group_iter),
                                         desc="point-wise metrics"):

    gt = g["GT"].iloc[0]
    preds = g.set_index("Quantile")["Prediction"]

    if 0.5 not in preds:          # need the median
        continue

    # point errors
    y_pred = preds[0.5]
    mae  = abs(y_pred - gt)
    mse  = (y_pred - gt) ** 2
    mape = abs((y_pred - gt) / gt) if gt else np.nan
    nmse = np.nan                # single point ⇒ var = 0

    # WIS for this timestamp
    qs  = sorted(q for q in preds.index if q != 0.5)
    n   = len(qs) // 2
    ae  = mae
    int_scores = 0
    for i in range(n):
        lo_q, hi_q = qs[i], qs[-(i + 1)]
        lo, hi     = preds[lo_q], preds[hi_q]
        alpha      = hi_q - lo_q
        int_scores += (hi - lo) \
                    + (2/alpha) * max(lo - gt, 0) \
                    + (2/alpha) * max(gt - hi, 0)

    wis = (ae + int_scores) / (1 + n)

    rows.append({
        "Unique_id"      : uid,
        "Reference Date" : ref_date,
        "Target End Date": tgt_date,
        "GT"             : gt,
        "Prediction"     : y_pred,
        "MAE"            : mae,
        "MSE"            : mse,
        "MAPE"           : mape,
        "NMSE"           : nmse,
        "WIS"            : wis
    })

pointwise_df = pd.DataFrame(rows).sort_values(
    ["Unique_id", "Reference Date", "Target End Date"])

point-wise metrics: 100%|██████████| 345600/345600 [02:57<00:00, 1949.93it/s]


In [17]:
pointwise_df

,Unique_id,Reference Date,Target End Date,GT,Prediction,MAE,MSE,MAPE,NMSE,WIS
0,Y_1,2011-01-08,2011-01-15,3.68098,3.579459,0.101521,0.010307,0.027580,NaN,3.728643
1,Y_1,2011-01-08,2011-01-22,4.43182,3.717173,0.714647,0.510721,0.161254,NaN,4.899479
2,Y_1,2011-01-08,2011-01-29,8.24176,4.270710,3.971050,15.769238,0.481821,NaN,18.096487
3,Y_1,2011-01-08,2011-02-05,6.80219,6.257584,0.544606,0.296596,0.080063,NaN,8.569783
4,Y_1,2011-01-15,2011-01-22,4.43182,3.775015,0.656805,0.431392,0.148202,NaN,5.108229
...,...,...,...,...,...,...,...,...,...,...
345595,Y_9999,1957-02-16,1957-03-16,0.00000,0.996371,0.996371,0.992755,NaN,NaN,5.149795
345596,Y_9999,1957-02-23,1957-03-02,0.00000,0.939883,0.939883,0.883380,NaN,NaN,4.944821
345597,Y_9999,1957-02-23,1957-03-09,0.00000,0.969965,0.969965,0.940832,NaN,NaN,5.076347
345598,Y_9999,1957-02-23,1957-03-16,0.00000,0.988516,0.988516,0.977164,NaN,NaN,5.146864


In [18]:
pointwise_df.to_csv('GRU_ALL_STATS.csv')